In [0]:
spark.conf.set(
    "fs.azure.account.key.projectnf.dfs.core.windows.net",
  "ntzj8hOFCbU6O7L069dR4Bxqnb/uE8v9STC+eqHFitEANjJpSskZE0oW9GI6o3xr0TP5AOagZb0P+AStupAZ8g=="
)


In [0]:
df = spark.read.format("delta").load(
    "abfss://lakehouse@projectnf.dfs.core.windows.net/gold/brain_metadata_clean/"
)

df.show(5)
df.count()


+---------+-----+--------------------+-----+-----------+
| filename|label|                path| size|label_index|
+---------+-----+--------------------+-----+-----------+
|1 no.jpeg|   no|abfss://lakehouse...|54521|          0|
|10 no.jpg|   no|abfss://lakehouse...| 3848|          0|
|11 no.jpg|   no|abfss://lakehouse...| 3475|          0|
|12 no.jpg|   no|abfss://lakehouse...| 4142|          0|
|13 no.jpg|   no|abfss://lakehouse...| 4570|          0|
+---------+-----+--------------------+-----+-----------+
only showing top 5 rows


506

In [0]:
from pyspark.sql.functions import col, when

# start from your clean df
df_feats = df.withColumn("size_kb", (col("size") / 1024.0))

df_feats = df_feats.withColumn(
    "size_bucket",
    when(col("size_kb") < 50,  "small")
     .when(col("size_kb") < 150, "medium")
     .otherwise("large")
)

df_feats.select("filename", "size", "size_kb", "size_bucket", "label_index").show(10)


+--------+-----+-------------+-----------+-----------+
|filename| size|      size_kb|size_bucket|label_index|
+--------+-----+-------------+-----------+-----------+
| Y52.jpg|61240|   59.8046875|     medium|          1|
| Y53.jpg| 5011| 4.8935546875|      small|          1|
| Y54.jpg|15874| 15.501953125|      small|          1|
| Y55.jpg|43087|42.0771484375|      small|          1|
| Y56.jpg| 7633| 7.4541015625|      small|          1|
| Y58.JPG|21237|20.7392578125|      small|          1|
| Y59.JPG|17890| 17.470703125|      small|          1|
|  Y6.jpg|86579|84.5498046875|     medium|          1|
| Y60.jpg| 7400|    7.2265625|      small|          1|
| Y61.jpg|65584|    64.046875|     medium|          1|
+--------+-----+-------------+-----------+-----------+
only showing top 10 rows


In [0]:
train_df, val_df, test_df = df_feats.randomSplit([0.7, 0.15, 0.15], seed=42)

train_df.count(), val_df.count(), test_df.count()


(361, 60, 85)

In [0]:
base = "abfss://lakehouse@projectnf.dfs.core.windows.net/gold/brain_features_v1"

train_df.write.format("delta").mode("overwrite").save(base + "/train")
val_df.write.format("delta").mode("overwrite").save(base + "/val")
test_df.write.format("delta").mode("overwrite").save(base + "/test")


In [0]:
train = spark.read.format("delta").load(base + "/train")
val   = spark.read.format("delta").load(base + "/val")
test  = spark.read.format("delta").load(base + "/test")

print("Train rows:", train.count())
print("Val rows  :", val.count())
print("Test rows :", test.count())

train.show(5)


Train rows: 361
Val rows  : 60
Test rows : 85
+---------+-----+--------------------+----+-----------+------------+-----------+
| filename|label|                path|size|label_index|     size_kb|size_bucket|
+---------+-----+--------------------+----+-----------+------------+-----------+
|35 no.jpg|   no|abfss://lakehouse...|6021|          0|5.8798828125|      small|
|37 no.jpg|   no|abfss://lakehouse...|6770|          0| 6.611328125|      small|
|38 no.jpg|   no|abfss://lakehouse...|7371|          0|7.1982421875|      small|
|40 no.jpg|   no|abfss://lakehouse...|5758|          0| 5.623046875|      small|
|41 no.jpg|   no|abfss://lakehouse...|5758|          0| 5.623046875|      small|
+---------+-----+--------------------+----+-----------+------------+-----------+
only showing top 5 rows


In [0]:
from pyspark.sql.functions import col

print("Train class counts:")
train.groupBy("label_index").count().show()

print("Val class counts:")
val.groupBy("label_index").count().show()

print("Test class counts:")
test.groupBy("label_index").count().show()

# Optional: size bucket distribution
print("Train size_bucket distribution:")
train.groupBy("size_bucket").count().show()


Train class counts:
+-----------+-----+
|label_index|count|
+-----------+-----+
|          1|  223|
|          0|  138|
+-----------+-----+

Val class counts:
+-----------+-----+
|label_index|count|
+-----------+-----+
|          1|   34|
|          0|   26|
+-----------+-----+

Test class counts:
+-----------+-----+
|label_index|count|
+-----------+-----+
|          1|   53|
|          0|   32|
+-----------+-----+

Train size_bucket distribution:
+-----------+-----+
|size_bucket|count|
+-----------+-----+
|     medium|   46|
|      small|  300|
|      large|   15|
+-----------+-----+



In [0]:
train.count(), val.count(), test.count()
train.groupBy("label_index").count().show()


+-----------+-----+
|label_index|count|
+-----------+-----+
|          1|  223|
|          0|  138|
+-----------+-----+



In [0]:
base = "abfss://lakehouse@projectnf.dfs.core.windows.net/gold/brain_features_v1"

train = spark.read.format("delta").load(base + "/train")
val   = spark.read.format("delta").load(base + "/val")
test  = spark.read.format("delta").load(base + "/test")


In [0]:
from pyspark.sql.functions import when, col

threshold = 10.0  

train_pred = train.withColumn(
    "prediction",
    when(col("size_kb") > threshold, 1).otherwise(0)
)

val_pred = val.withColumn(
    "prediction",
    when(col("size_kb") > threshold, 1).otherwise(0)
)

test_pred = test.withColumn(
    "prediction",
    when(col("size_kb") > threshold, 1).otherwise(0)
)


In [0]:
val_pred.groupBy("label_index", "prediction").count().show()


+-----------+----------+-----+
|label_index|prediction|count|
+-----------+----------+-----+
|          1|         0|    2|
|          1|         1|   32|
|          0|         0|   12|
|          0|         1|   14|
+-----------+----------+-----+



In [0]:
tp = val_pred.filter("prediction = 1 AND label_index = 1").count()
tn = val_pred.filter("prediction = 0 AND label_index = 0").count()
fp = val_pred.filter("prediction = 1 AND label_index = 0").count()
fn = val_pred.filter("prediction = 0 AND label_index = 1").count()

total = val_pred.count()

accuracy  = (tp + tn) / total if total > 0 else 0
precision = tp / (tp + fp) if (tp + fp) > 0 else 0
recall    = tp / (tp + fn) if (tp + fn) > 0 else 0

accuracy, precision, recall


(0.7333333333333333, 0.6956521739130435, 0.9411764705882353)

In [0]:
from pyspark.sql.functions import udf
from pyspark.sql.types import IntegerType

best_threshold = 10.0

def predict_tumor(size_kb):
    return 1 if size_kb > best_threshold else 0

predict_udf = udf(predict_tumor, IntegerType())


In [0]:
deployed_test = test.withColumn("prediction", predict_udf(col("size_kb")))
deployed_test.show(10)


+--------+-----+--------------------+-----+-----------+-------------+-----------+----------+
|filename|label|                path| size|label_index|      size_kb|size_bucket|prediction|
+--------+-----+--------------------+-----+-----------+-------------+-----------+----------+
|Y106.jpg|  yes|abfss://lakehouse...| 6155|          1| 6.0107421875|      small|         0|
|Y111.JPG|  yes|abfss://lakehouse...|25806|          1| 25.201171875|      small|         1|
|Y120.JPG|  yes|abfss://lakehouse...|15723|          1|15.3544921875|      small|         1|
|Y146.JPG|  yes|abfss://lakehouse...|24840|          1|   24.2578125|      small|         1|
|Y147.JPG|  yes|abfss://lakehouse...|28150|          1| 27.490234375|      small|         1|
| Y15.jpg|  yes|abfss://lakehouse...|17045|          1|16.6455078125|      small|         1|
|Y159.JPG|  yes|abfss://lakehouse...|22534|          1| 22.005859375|      small|         1|
|Y164.JPG|  yes|abfss://lakehouse...|17907|          1|17.4873046875| 

In [0]:
def metrics(df):
    tp = df.filter("prediction = 1 AND label_index = 1").count()
    tn = df.filter("prediction = 0 AND label_index = 0").count()
    fp = df.filter("prediction = 1 AND label_index = 0").count()
    fn = df.filter("prediction = 0 AND label_index = 1").count()

    total = df.count()

    accuracy  = (tp + tn) / total if total > 0 else 0
    precision = tp / (tp + fp)   if (tp + fp) > 0 else 0
    recall    = tp / (tp + fn)   if (tp + fn) > 0 else 0

    return accuracy, precision, recall

metrics(deployed_test)


(0.6823529411764706, 0.7321428571428571, 0.7735849056603774)

In [0]:
deployed_test.groupBy("prediction").count().show()


+----------+-----+
|prediction|count|
+----------+-----+
|         1|   56|
|         0|   29|
+----------+-----+

